# Nonlinear (np.cos) Regression with SGD

This notebook evaluates convergence behavior and generalization performance of SGD on synthetic cosine experimental data on our linear/polynomial regression model. We compare our three stepsize strategies — fixed, halving, and diminishing — using both training and test error over 20 randomized trials. We can observe how well a linear model class performs with nonlinearly (in this case, cosine) generated data.

In [1]:
import sys
sys.path.append("..") 

import numpy as np
import matplotlib.pyplot as plt
from SGD.sgd import SGD
from data.generate_synthetic_data import generate_training_data_unfixed
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

### Summary of Train/Test Error Over 20 Runs

We generate new synthetic data each time and report mean ± std of final losses.

In [ ]:
n_runs = 20
degree = 1
noise = 0.01
m, n = 100, 1

train_losses = {"fixed": [], "halving": [], "diminishing": []}
test_losses = {"fixed": [], "halving": [], "diminishing": []}

for run in range(n_runs):
    X, y, _ = generate_training_data_unfixed(
        m=m, n=n, noise=noise, model_type='nonlinear', nonlinear_func=np.cos
    )
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.3)

    poly = PolynomialFeatures(degree=degree, include_bias=False)
    X_train = poly.fit_transform(X_train_raw)
    X_test = poly.transform(X_test_raw)
    X_train /= np.max(np.abs(X_train), axis=0)
    X_test /= np.max(np.abs(X_test), axis=0)

    sgd = SGD(X_train, y_train, num_iterations=5000, noise=noise)

    for method in ["fixed", "halving", "diminishing"]:
        w, obj, grad, dist = sgd.optimize(stepsize_type=method)
        train_losses[method].append(obj[-1])

        test_pred = X_test @ w[1:] + w[0] 
        test_loss = np.mean((test_pred - y_test) ** 2)
        test_losses[method].append(test_loss)